In [1]:
import os
import pandas as pd
from itertools import repeat
import datetime
import math
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import os, sys
sys.path.insert(0, os.path.abspath('../'))


TOPDIR = [x for x in sys.path if 'RT-Glow' in x][0]
DATADIR = os.path.join(TOPDIR,'data/20201216_Lum_Image_Run')

#this reads in the re-ordered pyseg output csv file
cc = pd.read_csv(os.path.join(DATADIR,'cellcount20201214reorder.csv'))

cc.reset_index()



,index,﻿,file_name,cell_count,file_name_ch2,ch2_pos,plate_id,well
0,0,7953,C2_02_1_1_RFP_001.tif,65,C2_02_2_1_GFP_001.tif,2,2,C02
1,1,8067,C2_02_1_2_RFP_001.tif,115,C2_02_2_2_GFP_001.tif,1,2,C02
2,2,8145,C3_02_1_1_RFP_001.tif,124,C3_02_2_1_GFP_001.tif,4,2,C03
3,3,8289,C3_02_1_2_RFP_001.tif,157,C3_02_2_2_GFP_001.tif,3,2,C03
4,4,8184,C4_02_1_2_RFP_001.tif,50,C4_02_2_2_GFP_001.tif,0,2,C04
...,...,...,...,...,...,...,...,...
63355,63355,26359,N21_02_1_1_RFP_001.tif,75,N21_02_2_1_GFP_001.tif,24,240,N21
63356,63356,25897,N22_02_1_1_RFP_001.tif,49,N22_02_2_1_GFP_001.tif,25,240,N22
63357,63357,25908,N22_02_1_2_RFP_001.tif,59,N22_02_2_2_GFP_001.tif,16,240,N22
63358,63358,25918,N23_02_1_1_RFP_001.tif,83,N23_02_2_1_GFP_001.tif,36,240,N23


#### Momentum log file

In [2]:
ml_fn = 'MOMENTUM_LOGS_FOR_V004831B_20201214_100msRFP_10msGFP_20201216_153632_0.txt'
ml_path = os.path.join(DATADIR,ml_fn)


In [3]:
logDF = pd.read_table(ml_path, sep="\t")

ParserError: Error tokenizing data. C error: Expected 13 fields in line 6, saw 14


In [ ]:
#these next lines create series from summing ever two rows of the data or just taking the value from every other row in the wellcond case, this will work as long as the imaging tiles == 2 if it is any other number then the value will need to be changed

dd = pd.DataFrame()
ccgb = cc['cell_count'].groupby(cc.index // 2).sum()
gfpgb = cc['ch2_pos'].groupby(cc.index // 2).sum()
wellcond = cc.iloc[::2, 6]
wellcond = wellcond.reset_index()

logDF = pd.read_csv(os.path.join(DATADIR,'MOMENTUM_LOGS_20201216.csv'))

#truncated dataframe with just the columns with read, datetime, or barcode info

logDF_RDT = logDF.iloc[:,[5,6,11,13]]
logDF_RDT.columns = ['action','read_type','datetime','barcode']

#This dataframe now contains only the relevant information from the momentum log i.e. the rows where the action is read and the columns for datetime and barcode
logDF_RO = logDF_RDT.loc[logDF_RDT['action'] == 'Read']
logDF_RO.reset_index()

ReadTime = logDF_RO.iloc[::2, 2]
ReadTime.reset_index()

#since there are 264 reads per individual plate, this creates a list going over each read time 264 times
readtime_plate = []
for d in ReadTime:   
    for i in repeat(None, 264):
       readtime_plate.append(d)
#since there are 264 reads per individual plate, this creates a list going over each barcode 264 times
cell_line_list = ['WM88','H1048','H841','DMS53']
cll_264 = []
for c in cell_line_list:   
    for i in repeat(None, 264):
       cll_264.append(c)
cll_264=cll_264*30

#since there are 264 reads per individual plate, this creates a list going over each barcode 264 times
barcode_list = ['V004831B','V004832B','V004833B','V004834B']    
bll_264 = []
for b in barcode_list:   
    for i in repeat(None, 264):
       bll_264.append(b)
bll_264=bll_264*30       

read = [*range(1,31)]
read_264_4plates =[]
for r in read:   
    for i in repeat(None, 1056):
       read_264_4plates.append(r)
#there are 264 wells per plate
ccDF = pd.DataFrame()
ccDF['Cell_Count'] = ccgb
ccDF['ch2_pos'] = gfpgb
ccDF['Live_Dead']= ccDF['Cell_Count']-ccDF['ch2_pos']
ccDF['Well']=wellcond['well']
ccDF['Cell_Line']=cll_264
ccDF['Plate_Name']=bll_264
ccDF['Read_Number']=read_264_4plates
ccDF['DateTime_Image']=readtime_plate

ccDF['DateTime_Image']=pd.to_datetime(ccDF['DateTime_Image'], format='%m/%d/%y %H:%M')
tzeroimage = datetime.datetime(2020,12,16,14,0,0)
ccDFdtdif = (ccDF['DateTime_Image'] - tzeroimage)
ccDFdtdif = (ccDFdtdif / np.timedelta64(1, 's'))/3600
ccDFdtdif = pd.DataFrame(ccDFdtdif)
ccDFdtdif =ccDFdtdif.rename(columns={'DateTime_Image':'TotHour_Image'})
ccDF['TotHour_Image']=ccDFdtdif['TotHour_Image']

